In [1]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
#math tools
from itertools import combinations
import statistics
from statistics import mean
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.dates as dates
# Allows plots to appear directly in the notebook.
%matplotlib inline
# For dealing with some Accented characters (in Irish Place names)
import unidecode
# Date/time functionality
import datetime
import time
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
# look at some z-scores for inspecting outliers.
from scipy import stats
import seaborn as sb
# lookup lat/long and convert lat/long to national grid references.
#import geopy
#import pyproj

from patsy import dmatrices
from sklearn import metrics

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
#from sklearn.tree import export_text

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

#import graphviz
#from graphviz import Source
#to read all CSV files in a folder
import os
import glob

#### Functions generated at notebooks that might be useful

In [2]:
#method to read chunks from a file and create a dataframe list.
def get_chunks(x):
    """x=[location, dtypes, datecolumns ,date_parser_func, cols]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:cols=['Date time','Temperature','Wind Speed','Precipitation','Cloud Cover']
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser, cols]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    counter=0
    for filename in chunk_folder:
        # read the csv file
        if len(x)==4:
            df_chunk = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        else:
            df_chunk = pd.read_csv(filename,usecols=x[4], low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        chunk_list.append(df_chunk)
        print('Index at chunk_list',counter,' is ', filename, ': ',df_chunk.shape)
        counter+=1
    return chunk_list
#chunk_dbus_list=get_chunks(arg_list)

In [3]:
#method to parse dates
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")

In [4]:
#method that mathes list index in a dataframe list with id information in dataframe's column. 
#for this method to work id column should be constant. 
def id_list_index_matcher(df_list, column_name):
    df_id_list_index = pd.DataFrame()
    index_count=0
    for df in df_list:
        a=df[column_name].unique()
        df_id_list_index=df_id_list_index.append({'List_index':index_count,column_name:a[0]},ignore_index=True)
        index_count+=1
    return df_id_list_index

In [5]:
#methods that create csv files from dataframe list by naming them with unique value in id column.
#for this method to work id column should be constant. 
def write_to_csv(adress,df_list,id_column ):
    cwd=os.getcwd()
    path = cwd+adress
    for df in df_list:
        a=df[id_column].unique()
        file_name=a[0]
        saved_file_name=file_name+'.csv'
        my_file_bool=os.path.exists(path+saved_file_name)
        if my_file_bool==False:
            df.to_csv(path+saved_file_name,index=False)
            print('Location:', path+saved_file_name)

In [6]:
def write_to_csv_n_r_p(adress,df_list,id_column ):
    """without relative path input"""
    path = adress
    for df in df_list:
        a=df[id_column].unique()
        file_name=a[0]
        saved_file_name=file_name+'.csv'
        my_file_bool=os.path.exists(path+saved_file_name)
        if my_file_bool==False:
            df.to_csv(path+saved_file_name,index=False)
            print('Location:', path+saved_file_name)

In [7]:
def write_to_csv_if_in_list(adress,df_list,id_column,IMP_List ):
    cwd=os.getcwd()
    path = cwd+adress
    for df in df_list:
        a=df[id_column].unique()
        if a in IMP_List:
            file_name=a[0]
            saved_file_name=file_name+'.csv'
            my_file_bool=os.path.exists(path+saved_file_name)
            if my_file_bool==False:
                df.to_csv(path+saved_file_name,index=False)
                print('Location:', path+saved_file_name)

In [9]:
#method that takes a dataframe list and prints list lenght and sum of all rows in all dataframes
def df_list_checker(dflist):
    print("number of df in list: ",len(dflist))
    row_count=0
    for df in dflist:
        row_count+=df.shape[0]
    print("number of rows of data in list: ", row_count)

In [8]:
#method to yield(return generator object) dfs for memory efficent concat( to avoid expotential memory use)
#https://stackoverflow.com/questions/70911236/how-can-i-concat-the-huge-pandas-dataframes-into-one
def yield_dfs(x):
    """x=[location, dtypes, datecolumns ,date_parser_func]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    file_list = glob.glob(os.path.join(path, "*.csv"))
    """generator function to yield dataframes."""
    for filename in file_list:
        df = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3],index_col=False)
        print(df.shape)
        yield df

#df_dbus = pd.concat(yield_dfs(x), axis=0,ignore_index=True)

In [10]:
#method that takes a directory and reads all files inside that directory and its subdirectories 
#creates a dataframe from them and add to dataframe list
def get_chunks_subdirectory(location):
    cwd=os.getcwd()
    adress=location
    path = cwd+adress
    chunk_list=[]
    for root, dirnames, files in os.walk(path):
        print(dirnames)
        print('='*66)
        for mfile in files:
            # read the csv file
            df_chunk = pd.read_csv(os.path.join(root, mfile), low_memory=False)
            chunk_list.append(df_chunk)
            print('Location:', mfile)
    return chunk_list
#chunk_dbus_list_v2=get_chunks_subdirectory('/Chunks/DBUS/LINEID_CHUNKS/')

In [12]:
cols=['ROUTEID','PROGRNUMBER','TRIPID','PLANNEDTIME_DEP_R_M5','STOPPOINTID']
dtype={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','ROUTEID':'category','PLANNEDTIME_DEP_R_M5':'category'}
arg_list=['/Chunks/DBUS/IMP_ROUTEID_CHUNKS',dtype,[],mydateparser, cols]

In [13]:
dbus_df_list=get_chunks(arg_list)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/18_3.csv :  (826557, 5)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/67_6.csv :  (775062, 5)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/142_9.csv :  (23881, 5)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/17_16.csv :  (29666, 5)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/46A_74.csv :  (2094278, 5)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/122_14.csv :  (427306, 5)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/150_8.csv :  (524122, 5)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/33X_49.csv :  (56923, 5)
Index at chunk_list 8  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/83_19.csv :  (96394, 5)
Index at chu

Index at chunk_list 74  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/9_7.csv :  (1391972, 5)
Index at chunk_list 75  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/79A_28.csv :  (266196, 5)
Index at chunk_list 76  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/75_18.csv :  (86250, 5)
Index at chunk_list 77  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/142_12.csv :  (46872, 5)
Index at chunk_list 78  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/17A_17.csv :  (587575, 5)
Index at chunk_list 79  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/14_16.csv :  (1290699, 5)
Index at chunk_list 80  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/15A_83.csv :  (614112, 5)
Index at chunk_list 81  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/39_21.csv :  (971904, 5)
Index at chunk_list 82  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/151_18.csv :  (110262, 5

Index at chunk_list 148  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/66B_57.csv :  (85059, 5)
Index at chunk_list 149  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/239_26.csv :  (105100, 5)
Index at chunk_list 150  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/13_65.csv :  (131825, 5)
Index at chunk_list 151  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/84X_57.csv :  (40425, 5)
Index at chunk_list 152  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/54A_11.csv :  (407360, 5)
Index at chunk_list 153  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/47_139.csv :  (263370, 5)
Index at chunk_list 154  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/41C_84.csv :  (56880, 5)
Index at chunk_list 155  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/185_53.csv :  (40228, 5)
Index at chunk_list 156  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/9_5.csv :  (13

Index at chunk_list 222  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/33_70.csv :  (410226, 5)
Index at chunk_list 223  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/238_11.csv :  (98449, 5)
Index at chunk_list 224  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/63_25.csv :  (73410, 5)
Index at chunk_list 225  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/104_16.csv :  (135076, 5)
Index at chunk_list 226  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/69_45.csv :  (176777, 5)
Index at chunk_list 227  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/41_22.csv :  (51534, 5)
Index at chunk_list 228  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/17A_15.csv :  (590993, 5)
Index at chunk_list 229  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/185_51.csv :  (8122, 5)
Index at chunk_list 230  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/68_66.csv :  (3061

Index at chunk_list 296  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/39_20.csv :  (802676, 5)
Index at chunk_list 297  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/63_26.csv :  (230720, 5)
Index at chunk_list 298  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/66B_58.csv :  (161625, 5)
Index at chunk_list 299  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/17_10.csv :  (508784, 5)
Index at chunk_list 300  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/17A_14.csv :  (33313, 5)
Index at chunk_list 301  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/66A_38.csv :  (140232, 5)
Index at chunk_list 302  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/15A_85.csv :  (116290, 5)
Index at chunk_list 303  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/47_136.csv :  (259213, 5)
Index at chunk_list 304  is  /home/team8/notebook/Pelin/Chunks/DBUS/IMP_ROUTEID_CHUNKS/40D_103.csv :

In [14]:
dbus_df_list[0].head()

,TRIPID,PROGRNUMBER,STOPPOINTID,ROUTEID,PLANNEDTIME_DEP_R_M5
0,5970352,1,4359,18_3,09:30
1,5970352,2,4360,18_3,09:30
2,5970352,3,4361,18_3,09:30
3,5970352,4,2202,18_3,09:30
4,5970352,5,2203,18_3,09:35


Calculating sum of tripid counts in all chunks

In [15]:
total_tripid_count=0
for df in dbus_df_list:
    total_tripid_count+=df['TRIPID'].nunique()

In [17]:
total_tripid_count

636247

In [18]:
dbus_df_list[0].shape[0]

826557

In [20]:
Num_uniq_trip_id=dbus_df_list[0]['TRIPID'].nunique()
Num_uniq_trip_id

3904

In [21]:
ROUTEID_vs_weights=pd.DataFrame()
for df in dbus_df_list:
    RouteID=df['ROUTEID'].unique()[0]
    Num_uniq_trip_id=df['TRIPID'].nunique()
    mydata=[[RouteID,Num_uniq_trip_id/total_tripid_count]]
    row_df=pd.DataFrame( mydata, columns=['ROUTEID', 'Weight'])
    ROUTEID_vs_weights=pd.concat([ROUTEID_vs_weights,row_df],axis=0,ignore_index=True)
ROUTEID_vs_weights.head(5)

,ROUTEID,Weight
0,18_3,0.006136
1,67_6,0.006391
2,142_9,0.000396
3,17_16,0.000483
4,46A_74,0.017811


In [22]:
sub_dbus_df_list=[]
for df in dbus_df_list:
    print('='*66)
    print("ROUTEID is ",df['ROUTEID'].unique()[0])
    print('_'*66)
    df_max_PROGRNUMBER=pd.DataFrame(df.groupby(['TRIPID'])['PROGRNUMBER'].max())
    df_max_PROGRNUMBER.rename(columns={'PROGRNUMBER':'PROGRNUMBER_MAX'},inplace=True)
    df=pd.merge( df, df_max_PROGRNUMBER, on="TRIPID", how="left")
    max_PROGRNUMBER= np.amax(df['PROGRNUMBER'].unique() )
    df.drop(df[df.PROGRNUMBER_MAX != max_PROGRNUMBER].index, inplace=True)
    target_series=df.groupby(['ROUTEID','STOPPOINTID'])['PROGRNUMBER'].unique()
    target=pd.DataFrame({'PROGRNUMBER':target_series}).reset_index()
    print(target)
    sub_dbus_df_list.append(target)


ROUTEID is  18_3
__________________________________________________________________
   ROUTEID STOPPOINTID PROGRNUMBER
0     18_3        1421        [26]
1     18_3        2181        [22]
2     18_3        2182        [23]
3     18_3        2185        [25]
4     18_3        2202         [4]
..     ...         ...         ...
59    18_3         779        [52]
60    18_3         780        [53]
61    18_3         781        [54]
62    18_3         903        [50]
63    18_3         904        [51]

[64 rows x 3 columns]
ROUTEID is  67_6
__________________________________________________________________
   ROUTEID STOPPOINTID PROGRNUMBER
0     67_6        1472        [53]
1     67_6        1473        [55]
2     67_6        1474        [56]
3     67_6        1476        [57]
4     67_6        1478        [59]
..     ...         ...         ...
60    67_6        7074         [4]
61    67_6        7108        [15]
62    67_6        7110        [18]
63    67_6        7385        [65]
64  

   ROUTEID STOPPOINTID PROGRNUMBER
0    84_30        2995        [74]
1    84_30        3045        [44]
2    84_30        3140        [59]
3    84_30        3141        [60]
4    84_30        3142        [61]
..     ...         ...         ...
81   84_30        5136         [4]
82   84_30        7188        [86]
83   84_30        7271         [3]
84   84_30        7296         [8]
85   84_30        7653        [85]

[86 rows x 3 columns]
ROUTEID is  130_10
__________________________________________________________________
   ROUTEID STOPPOINTID PROGRNUMBER
0   130_10        1740        [13]
1   130_10        1741        [14]
2   130_10        1742        [15]
3   130_10        1743        [16]
4   130_10        1744        [17]
5   130_10        1745        [18]
6   130_10        1746        [19]
7   130_10        1747        [20]
8   130_10        1748        [21]
9   130_10        1749        [22]
10  130_10        1750        [29]
11  130_10        1751        [23]
12  130_10      

   ROUTEID STOPPOINTID PROGRNUMBER
0    26_28        1444         [9]
1    26_28        1445        [10]
2    26_28        1449        [12]
3    26_28        1450        [13]
4    26_28        1451        [14]
5    26_28        2191        [15]
6    26_28        2192        [16]
7    26_28        2193        [17]
8    26_28        2194        [18]
9    26_28        2195        [19]
10   26_28        2196        [20]
11   26_28        2197        [21]
12   26_28        2198        [22]
13   26_28        2200        [23]
14   26_28        2201        [24]
15   26_28        2202        [27]
16   26_28        2203        [28]
17   26_28        2210        [29]
18   26_28         312         [8]
19   26_28         317         [7]
20   26_28         346         [6]
21   26_28         400         [5]
22   26_28        4401        [26]
23   26_28         493         [2]
24   26_28         494         [3]
25   26_28         495         [4]
26   26_28        7078        [11]
27   26_28        72

   ROUTEID STOPPOINTID PROGRNUMBER
0    68_79        1278         [4]
1    68_79        1282         [5]
2    68_79        1284         [7]
3    68_79        1285         [8]
4    68_79        1286         [9]
..     ...         ...         ...
68   68_79        6119        [66]
69   68_79        6244        [49]
70   68_79        7483        [59]
71   68_79        7546         [2]
72   68_79        7547         [3]

[73 rows x 3 columns]
ROUTEID is  120_13
__________________________________________________________________
   ROUTEID STOPPOINTID PROGRNUMBER
0   120_13         263        [23]
1   120_13         284         [1]
2   120_13          30         [6]
3   120_13        4501        [13]
4   120_13        4502        [15]
5   120_13        4504         [2]
6   120_13        4935        [14]
7   120_13        4936        [16]
8   120_13        5147         [4]
9   120_13        6071        [24]
10  120_13          68         [5]
11  120_13          69         [7]
12  120_13      

   ROUTEID STOPPOINTID PROGRNUMBER
0   77A_29        1406        [16]
1   77A_29        1407        [17]
2   77A_29        1409        [18]
3   77A_29        1934         [8]
4   77A_29        2094        [15]
..     ...         ...         ...
68  77A_29        4931        [72]
69  77A_29        5008        [52]
70  77A_29        5133        [39]
71  77A_29        7459        [73]
72  77A_29        7460        [70]

[73 rows x 3 columns]
ROUTEID is  44_40
__________________________________________________________________
   ROUTEID STOPPOINTID PROGRNUMBER
0    44_40        1013        [29]
1    44_40        1015        [30]
2    44_40         119        [11]
3    44_40         207         [2]
4    44_40         208         [3]
..     ...         ...         ...
77   44_40         843        [28]
78   44_40         852        [34]
79   44_40         853        [35]
80   44_40         854        [36]
81   44_40         855        [37]

[82 rows x 3 columns]
ROUTEID is  47_138
__________

...

   ROUTEID STOPPOINTID PROGRNUMBER
0    27A_5        1219        [23]
1    27A_5        1220        [24]
2    27A_5        1221        [25]
3    27A_5        1277        [22]
4    27A_5        1791         [1]
5    27A_5        1792         [2]
6    27A_5        1793         [3]
7    27A_5        1796        [16]
8    27A_5        1797        [17]
9    27A_5        1798        [18]
10   27A_5        1799        [19]
11   27A_5        1800        [20]
12   27A_5        1801        [21]
13   27A_5         614        [31]
14   27A_5         615        [32]
15   27A_5         616        [33]
16   27A_5         617        [34]
17   27A_5         618        [35]
18   27A_5         619        [36]
19   27A_5         620        [38]
20   27A_5         635         [5]
21   27A_5         636         [6]
22   27A_5         637         [7]
23   27A_5         638         [8]
24   27A_5         639         [9]
25   27A_5         640        [10]
26   27A_5         641        [11]
27   27A_5         6

   ROUTEID STOPPOINTID PROGRNUMBER
0    49_28        1155        [21]
1    49_28        1157        [22]
2    49_28        1158        [23]
3    49_28        1159        [24]
4    49_28        1160        [25]
5    49_28        1161        [26]
6    49_28        1162        [27]
7    49_28        1163        [28]
8    49_28        1337        [29]
9    49_28        1338        [30]
10   49_28        1339        [31]
11   49_28        1340        [32]
12   49_28        1341        [33]
13   49_28        1342        [34]
14   49_28        1343        [35]
15   49_28        1344        [36]
16   49_28        1345        [37]
17   49_28        1347        [38]
18   49_28        1358        [45]
19   49_28        2002        [44]
20   49_28        2383        [43]
21   49_28        2539         [7]
22   49_28        2540         [8]
23   49_28        2542         [9]
24   49_28        2547        [18]
25   49_28        2548        [19]
26   49_28        2634        [39]
27   49_28        26

   ROUTEID STOPPOINTID PROGRNUMBER
0    66_18        1472        [41]
1    66_18        1473        [43]
2    66_18        1474        [44]
3    66_18        1476        [45]
4    66_18        1478        [47]
5    66_18        1479        [48]
6    66_18        2232        [22]
7    66_18        2233        [21]
8    66_18        2234        [23]
9    66_18        2235        [24]
10   66_18        2236        [25]
11   66_18        2237        [26]
12   66_18        2238        [27]
13   66_18        2239        [29]
14   66_18        2241        [30]
15   66_18        2242        [31]
16   66_18        2243        [32]
17   66_18        2244        [33]
18   66_18        2245        [34]
19   66_18        2246        [35]
20   66_18        2247        [36]
21   66_18        2248        [37]
22   66_18        2249        [38]
23   66_18        2250        [39]
24   66_18        2251        [40]
25   66_18        2810        [52]
26   66_18         315        [49]
27   66_18        33

   ROUTEID STOPPOINTID PROGRNUMBER
0   41C_79         119        [50]
1   41C_79        1631        [33]
2   41C_79        1632        [34]
3   41C_79        1633        [36]
4   41C_79        1634        [37]
..     ...         ...         ...
56  41C_79        6015        [24]
57  41C_79         733        [23]
58  41C_79        7593        [31]
59  41C_79        7603        [52]
60  41C_79        7655         [3]

[61 rows x 3 columns]
ROUTEID is  83A_20
__________________________________________________________________
   ROUTEID STOPPOINTID PROGRNUMBER
0   83A_20        1016        [38]
1   83A_20        1017        [39]
2   83A_20        1018        [40]
3   83A_20        1019        [41]
4   83A_20        1020        [42]
..     ...         ...         ...
56  83A_20        7132         [1]
57  83A_20        7423        [10]
58  83A_20        7424         [9]
59  83A_20        7425         [8]
60  83A_20        7458         [7]

[61 rows x 3 columns]
ROUTEID is  39_20
__________

   ROUTEID STOPPOINTID PROGRNUMBER
0   47_136        2066        [20]
1   47_136        2067        [21]
2   47_136        2068        [24]
3   47_136        2069        [25]
4   47_136        2070        [26]
5   47_136        2084        [27]
6   47_136        2085        [29]
7   47_136        2086        [30]
8   47_136        3181        [12]
9   47_136        3321        [19]
10  47_136        3485         [4]
11  47_136        3486         [5]
12  47_136        3487         [6]
13  47_136         371        [43]
14  47_136         381        [32]
15  47_136         382        [33]
16  47_136         383        [35]
17  47_136         384        [36]
18  47_136         385        [37]
19  47_136         387        [39]
20  47_136         388        [40]
21  47_136         389        [41]
22  47_136         391        [44]
23  47_136         392        [45]
24  47_136         393        [42]
25  47_136         395        [46]
26  47_136         396        [47]
27  47_136         3

In [23]:
index_tracker=0
for df in sub_dbus_df_list:
    print('='*66)
    print("ROUTEID is ",df['ROUTEID'].unique()[0]," list index ",index_tracker )
    print('_'*66)
    df['PROGRNUMBER']=df['PROGRNUMBER'].fillna(0)
    df['PROGRNUMBER']=df['PROGRNUMBER'].apply(lambda x : x[0] if type(x)==np.ndarray else int(x))
    print(df)
    index_tracker+=1


ROUTEID is  18_3  list index  0
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0     18_3        1421           26
1     18_3        2181           22
2     18_3        2182           23
3     18_3        2185           25
4     18_3        2202            4
..     ...         ...          ...
59    18_3         779           52
60    18_3         780           53
61    18_3         781           54
62    18_3         903           50
63    18_3         904           51

[64 rows x 3 columns]
ROUTEID is  67_6  list index  1
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0     67_6        1472           53
1     67_6        1473           55
2     67_6        1474           56
3     67_6        1476           57
4     67_6        1478           59
..     ...         ...          ...
60    67_6        7074            4
61    67_6        7108           15
62    67_6        7110 

23  120_13         831           11
ROUTEID is  83_25  list index  29
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    83_25        1069           19
1    83_25        1070           20
2    83_25        1071           21
3    83_25        1072           23
4    83_25        1170           18
..     ...         ...          ...
58   83_25        7131           63
59   83_25        7426           53
60   83_25        7427           54
61   83_25        7428           55
62   83_25        7430           48

[63 rows x 3 columns]
ROUTEID is  16_21  list index  30
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    16_21         119           12
1    16_21        1279           25
2    16_21        1282           26
3    16_21        1284           28
4    16_21        1285           29
..     ...         ...          ...
58   16_21        5052           63
59   16_21         

   ROUTEID STOPPOINTID  PROGRNUMBER
0    83_16        1016           45
1    83_16        1017           46
2    83_16        1018           47
3    83_16        1019           48
4    83_16        1020           49
..     ...         ...          ...
63   83_16        7424           15
64   83_16        7425           14
65   83_16        7431           21
66   83_16        7458           13
67   83_16        7500            2

[68 rows x 3 columns]
ROUTEID is  185_59  list index  43
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0   185_59        2113           20
1   185_59        4095            7
2   185_59        4108            1
3   185_59        4109            2
4   185_59        4110            3
5   185_59        4111            4
6   185_59        4112            5
7   185_59        4113            6
8   185_59        4131           21
9   185_59        4156           23
10  185_59        4167           24
11  185_59 

47   111_8        7647           16
ROUTEID is  84A_1  list index  68
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    84A_1        3082            2
1    84A_1        3114            3
2    84A_1        3115            4
3    84A_1        3116            5
4    84A_1        3117            6
5    84A_1        3118            7
6    84A_1        3119            8
7    84A_1        3120            9
8    84A_1        3122           10
9    84A_1        3123           11
10   84A_1        3124           12
11   84A_1        3125           13
12   84A_1        3126           14
13   84A_1        3127           15
14   84A_1        3128           16
15   84A_1        3130           18
16   84A_1        3131           19
17   84A_1        3132           20
18   84A_1        3133           21
19   84A_1        3134           25
20   84A_1        3135           26
21   84A_1        3136           27
22   84A_1        3138           28

54    7_73        7643           46
ROUTEID is  69_47  list index  86
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    69_47        1358           61
1    69_47        1471           52
2    69_47        1472           53
3    69_47        1473           55
4    69_47        1474           56
..     ...         ...          ...
57   69_47        6002           17
58   69_47        7432           10
59   69_47        7435           50
60   69_47        7453           58
61   69_47        7665           62

[62 rows x 3 columns]
ROUTEID is  13_66  list index  87
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    13_66         107            8
1    13_66         110           11
2    13_66         112           12
3    13_66         113           13
4    13_66         114           14
..     ...         ...          ...
83   13_66        7092           10
84   13_66        7

...

50    41_3          85           13
ROUTEID is  75_19  list index  205
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    75_19        1138           61
1    75_19        1139           62
2    75_19        1140           63
3    75_19        1281           56
4    75_19        1317           53
..     ...         ...          ...
80   75_19        5129           59
81   75_19        5130           60
82   75_19        7181           85
83   75_19        7393           29
84   75_19        7658           12

[85 rows x 3 columns]
ROUTEID is  27_17  list index  206
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    27_17        1196           65
1    27_17        1197           66
2    27_17        1198           67
3    27_17        1199           68
4    27_17        1200           69
..     ...         ...          ...
85   27_17         672           63
86   27_17       

21  238_11        7505            7
ROUTEID is  63_25  list index  224
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    63_25        2039            1
1    63_25        2040            2
2    63_25        2041            3
3    63_25        2042            4
4    63_25        2043            5
5    63_25        2044            6
6    63_25        2045            7
7    63_25        2046            8
8    63_25        2047            9
9    63_25        2048           10
10   63_25        2049           11
11   63_25        2050           12
12   63_25        2051           13
13   63_25        2053           14
14   63_25        2054           15
15   63_25        2055           16
16   63_25        3126           23
17   63_25        3254           20
18   63_25        3255           24
19   63_25        3276           28
20   63_25        3277           36
21   63_25        3278           37
22   63_25        3279           3

45  66B_59        7453           39
ROUTEID is  37_15  list index  249
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0    37_15        1478           50
1    37_15        1479           51
2    37_15        1528           42
3    37_15        1680           12
4    37_15        1681           13
5    37_15        1682           14
6    37_15        1683           15
7    37_15        1684           16
8    37_15        1685           17
9    37_15        1686           18
10   37_15        1687           19
11   37_15        1688           20
12   37_15        1689           21
13   37_15        1690           22
14   37_15        1691           23
15   37_15        1692           24
16   37_15        1693           25
17   37_15        1694           26
18   37_15        1695           27
19   37_15        1696           28
20   37_15        1697           29
21   37_15        1698           30
22   37_15        1699           3

   ROUTEID STOPPOINTID  PROGRNUMBER
0   77A_28        1358           63
1   77A_28        1424           48
2   77A_28        1436           53
3   77A_28        2002           62
4   77A_28        2183           44
..     ...         ...          ...
64  77A_28        4933            5
65  77A_28        4960            3
66  77A_28        5099           60
67  77A_28        7043           47
68  77A_28        7550           14

[69 rows x 3 columns]
ROUTEID is  40E_91  list index  269
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0   40E_91        1582           15
1   40E_91        1584           18
2   40E_91        1585           19
3   40E_91        1586           20
4   40E_91        1600           21
5   40E_91          30           25
6   40E_91        4793           16
7   40E_91        5147           23
8   40E_91        6068           17
9   40E_91          68           24
10  40E_91          69           26
11  40E_91

   ROUTEID STOPPOINTID  PROGRNUMBER
0   77A_31        1358           63
1   77A_31        1424           48
2   77A_31        1436           53
3   77A_31        2002           62
4   77A_31        2183           44
..     ...         ...          ...
64  77A_31        4933            5
65  77A_31        4960            3
66  77A_31        5099           60
67  77A_31        7043           47
68  77A_31        7550           14

[69 rows x 3 columns]
ROUTEID is  120_12  list index  292
__________________________________________________________________
   ROUTEID STOPPOINTID  PROGRNUMBER
0   120_12         192            3
1   120_12         276           26
2   120_12           3            2
3   120_12        4481           11
4   120_12        4485           12
5   120_12        4499            8
6   120_12        4500           10
7   120_12        4538           23
8   120_12        4934            9
9   120_12        5144           21
10  120_12        5145           25
11  120_12

In [ ]:
sub_dbus_df_list[0]['PROGRNUMBER'][0]

In [ ]:
type(sub_dbus_df_list[35]['PROGRNUMBER'][43])

In [ ]:
sub_dbus_df = pd.concat(sub_dbus_df_list, axis=0,ignore_index=True)

In [ ]:
sub_dbus_df 

In [ ]:
sub_dbus_df.shape

In [ ]:
sub_dbus_df.to_csv('routeid_vs_sp.csv',index=False) 

In [ ]:
dtypes={'ROUTEID':'category','STOPPOINTID':'category','PROGRNUMBER':'int16'}

In [ ]:
sub_dbus_df = pd.read_csv(routeid_vs_sp.csv, low_memory=False, dtype=dtypes)

In [ ]:
algotrithm_test_list=[x for x in range(1,66)]

In [ ]:
#https://stackoverflow.com/questions/25650827/get-all-ordered-combinations-of-list-python
numbers = algotrithm_test_list
for item in combinations(numbers, 2):
    print(sorted(item), end='')

In [ ]:
sub_dbus_df_list[1].shape

In [ ]:
test_df=sub_dbus_df_list[1].sort_values(by=['PROGRNUMBER'])
test_df

In [ ]:
combinations_array=[]
for item in combinations(test_df['PROGRNUMBER'], 2):
    a=sorted(item)
    combinations_array.append(a)
combinations_array

In [ ]:
len(combinations_array)

In [ ]:
#combinations_array[index_in_array][beginning_stop][locator_insubarray]
Beginning_stop=combinations_array[0][0][0]
print("Beginning stop", Beginning_stop)
#combinations_array[index_in_array][beginning_stop][locator_insubarray]
Ending_stop=combinations_array[0][1][0]
print("Ending stop",Ending_stop)

In [ ]:
Beginning_stop_series=test_df['STOPPOINTID'][test_df.PROGRNUMBER == Beginning_stop ].values
Ending_stop_series=test_df['STOPPOINTID'][test_df.PROGRNUMBER == Ending_stop ].values
Beginning_stop_series[0]

In [ ]:
Ending_stop_series[0]

In [ ]:
routeid=test_df['ROUTEID'].unique()[0]
data=[[routeid,Beginning_stop_series[0],Ending_stop_series[0] ]]
comb_test_df=pd.DataFrame( data, columns=['ROUTEID', 'Beginning_stop','Ending_stop'])
comb_test_df

In [ ]:
sub_dbus_df_list[1]['PROGRNUMBER'][1][0]

In [ ]:
atest=sub_dbus_df_list[1]['PROGRNUMBER']
atest.apply(lambda x : x[0])

In [35]:
weight_routeid=(ROUTEID_vs_weights['Weight'][ROUTEID_vs_weights.ROUTEID ==sub_dbus_df_list[1]['ROUTEID'].unique()[0] ].values)[0]

In [36]:
weight_routeid

0.006898397676691592

In [24]:
stop_point_pair_list=[]
for df in sub_dbus_df_list:
    routeid=df['ROUTEID'].unique()[0]
    print('='*66)
    print("ROUTEID is ",routeid)
    print('_'*66)
    df=df.sort_values(by=['PROGRNUMBER'])
    combinations_array=[]
    for item in combinations(df['PROGRNUMBER'], 2):
        a=sorted(item)
        combinations_array.append(a)
    index_counter=0
    target_df=pd.DataFrame()
    for pairs in combinations_array:
        #combinations_array[index_in_array][beginning_stop][locator_insubarray]
        Beginning_stop=combinations_array[index_counter][0]
        #combinations_array[index_in_array][ending_stop][locator_insubarray]
        Ending_stop=combinations_array[index_counter][1]
        index_counter+=1
        Beginning_stop_series=df['STOPPOINTID'][df.PROGRNUMBER == Beginning_stop ].values
        Ending_stop_series=df['STOPPOINTID'][df.PROGRNUMBER == Ending_stop ].values
        weight_routeid=(ROUTEID_vs_weights['Weight'][ROUTEID_vs_weights.ROUTEID ==routeid  ].values)[0]
        data=[[routeid,Beginning_stop_series[0],Ending_stop_series[0],weight_routeid ]]
        row_df=pd.DataFrame( data, columns=['ROUTEID', 'Beginning_stop','Ending_stop','Weight'])
        target_df=pd.concat([target_df,row_df],axis=0,ignore_index=True)
    stop_point_pair_list.append(target_df)

ROUTEID is  18_3
__________________________________________________________________
ROUTEID is  67_6
__________________________________________________________________
ROUTEID is  142_9
__________________________________________________________________
ROUTEID is  17_16
__________________________________________________________________
ROUTEID is  46A_74
__________________________________________________________________
ROUTEID is  122_14
__________________________________________________________________
ROUTEID is  150_8
__________________________________________________________________
ROUTEID is  33X_49
__________________________________________________________________
ROUTEID is  83_19
__________________________________________________________________
ROUTEID is  270_44
__________________________________________________________________
ROUTEID is  84_30
__________________________________________________________________
ROUTEID is  130_10
____________________________________________

ROUTEID is  46A_67
__________________________________________________________________
ROUTEID is  83A_26
__________________________________________________________________
ROUTEID is  17_15
__________________________________________________________________
ROUTEID is  1_38
__________________________________________________________________
ROUTEID is  27B_23
__________________________________________________________________
ROUTEID is  13_72
__________________________________________________________________
ROUTEID is  27B_40
__________________________________________________________________
ROUTEID is  83A_17
__________________________________________________________________
ROUTEID is  33_45
__________________________________________________________________
ROUTEID is  15_17
__________________________________________________________________
ROUTEID is  123_36
__________________________________________________________________
ROUTEID is  40D_101
________________________________________

ROUTEID is  185_60
__________________________________________________________________
ROUTEID is  26_29
__________________________________________________________________
ROUTEID is  114_6
__________________________________________________________________
ROUTEID is  84X_62
__________________________________________________________________
ROUTEID is  7A_86
__________________________________________________________________
ROUTEID is  270_42
__________________________________________________________________
ROUTEID is  7A_89
__________________________________________________________________
ROUTEID is  111_7
__________________________________________________________________
ROUTEID is  185_50
__________________________________________________________________
ROUTEID is  26_24
__________________________________________________________________
ROUTEID is  40_32
__________________________________________________________________
ROUTEID is  84_28
___________________________________________

ROUTEID is  59_11
__________________________________________________________________
ROUTEID is  120_7
__________________________________________________________________
ROUTEID is  151_17
__________________________________________________________________
ROUTEID is  70_60
__________________________________________________________________
ROUTEID is  27B_35
__________________________________________________________________
ROUTEID is  25A_270
__________________________________________________________________
ROUTEID is  32_57
__________________________________________________________________
ROUTEID is  41B_53
__________________________________________________________________
ROUTEID is  53_23
__________________________________________________________________
ROUTEID is  44_36
__________________________________________________________________
ROUTEID is  61_106
__________________________________________________________________
ROUTEID is  41C_78
________________________________________

ROUTEID is  122_18
__________________________________________________________________
ROUTEID is  7B_91
__________________________________________________________________
ROUTEID is  68_78
__________________________________________________________________
ROUTEID is  66A_34
__________________________________________________________________
ROUTEID is  27_12
__________________________________________________________________
ROUTEID is  7A_85
__________________________________________________________________
ROUTEID is  33_70
__________________________________________________________________
ROUTEID is  238_11
__________________________________________________________________
ROUTEID is  63_25
__________________________________________________________________
ROUTEID is  104_16
__________________________________________________________________
ROUTEID is  69_45
__________________________________________________________________
ROUTEID is  41_22
___________________________________________

ROUTEID is  13_61
__________________________________________________________________
ROUTEID is  7A_88
__________________________________________________________________
ROUTEID is  41C_80
__________________________________________________________________
ROUTEID is  185_55
__________________________________________________________________
ROUTEID is  27A_5
__________________________________________________________________
ROUTEID is  39X_1
__________________________________________________________________
ROUTEID is  37_14
__________________________________________________________________
ROUTEID is  56A_31
__________________________________________________________________
ROUTEID is  25_272
__________________________________________________________________
ROUTEID is  49_28
__________________________________________________________________
ROUTEID is  63_30
__________________________________________________________________
ROUTEID is  45A_70
__________________________________________

In [26]:
stop_point_pair_list[0]

,ROUTEID,Beginning_stop,Ending_stop,Weight
0,18_3,4359,4360,0.006136
1,18_3,4359,4361,0.006136
2,18_3,4359,2202,0.006136
3,18_3,4359,2203,0.006136
4,18_3,4359,2204,0.006136
...,...,...,...,...
2011,18_3,2800,7599,0.006136
2012,18_3,2800,375,0.006136
2013,18_3,2801,7599,0.006136
2014,18_3,2801,375,0.006136


In [27]:
write_to_csv('/RandomForestRegPickles/',stop_point_pair_list,'ROUTEID' )

Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/18_3.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/67_6.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/142_9.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/17_16.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/46A_74.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/122_14.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/150_8.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/33X_49.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/83_19.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/270_44.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/84_30.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/130_10.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/66A_36.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/46A_66.csv
Location: /home

Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/15B_61.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/142_8.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/27B_34.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/70_67.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/44B_61.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/45A_64.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/41C_83.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/161_51.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/33X_46.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/38_15.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/68_80.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/41B_52.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/13_73.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/184_29.csv
Location: /

Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/13_67.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/66_17.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/31_15.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/31A_26.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/25D_277.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/43_88.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/145_102.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/77A_28.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/40E_91.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/13_61.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/7A_88.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/41C_80.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/185_55.csv
Location: /home/team8/notebook/Pelin/stop_point_pair_chunks/27A_5.csv
Location: /

In [28]:
dtypes={'ROUTEID' :'category','Beginning_stop':'category','Ending_stop':'category'}
x=['/stop_point_pair_chunks',dtypes,[],mydateparser]

In [29]:
stop_point_pair_list= get_chunks(x)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/18_3.csv :  (2016, 4)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/67_6.csv :  (2080, 4)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/142_9.csv :  (136, 4)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/17_16.csv :  (2080, 4)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/46A_74.csv :  (1711, 4)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/122_14.csv :  (1225, 4)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/150_8.csv :  (595, 4)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/33X_49.csv :  (1485, 4)
Index at chunk_list 8  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/83_19.csv :  (1891, 4)
Index at chunk_list 9  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/270_44.csv :  (153, 4)
I

Index at chunk_list 112  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/7A_86.csv :  (1891, 4)
Index at chunk_list 113  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/270_42.csv :  (136, 4)
Index at chunk_list 114  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/7A_89.csv :  (2080, 4)
Index at chunk_list 115  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/111_7.csv :  (1035, 4)
Index at chunk_list 116  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/185_50.csv :  (190, 4)
Index at chunk_list 117  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/26_24.csv :  (406, 4)
Index at chunk_list 118  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/40_32.csv :  (946, 4)
Index at chunk_list 119  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/84_28.csv :  (4005, 4)
Index at chunk_list 120  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/17A_11.csv :  (1830, 4)
Index at chunk_list 121  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/7_74

Index at chunk_list 200  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/68_84.csv :  (2485, 4)
Index at chunk_list 201  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/7_72.csv :  (1485, 4)
Index at chunk_list 202  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/43_84.csv :  (1275, 4)
Index at chunk_list 203  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/69_43.csv :  (1596, 4)
Index at chunk_list 204  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/41_3.csv :  (1275, 4)
Index at chunk_list 205  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/75_19.csv :  (3570, 4)
Index at chunk_list 206  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/27_17.csv :  (4005, 4)
Index at chunk_list 207  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/33A_25.csv :  (1953, 4)
Index at chunk_list 208  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/47_135.csv :  (1485, 4)
Index at chunk_list 209  is  /home/team8/notebook/Pelin/stop_point_pair_chunks/76_

In [30]:
stop_point_pair_list[300]

,ROUTEID,Beginning_stop,Ending_stop,Weight
0,17A_14,956,957,0.000663
1,17A_14,956,958,0.000663
2,17A_14,956,959,0.000663
3,17A_14,956,960,0.000663
4,17A_14,956,961,0.000663
...,...,...,...,...
1706,17A_14,6180,7389,0.000663
1707,17A_14,6180,7025,0.000663
1708,17A_14,7297,7389,0.000663
1709,17A_14,7297,7025,0.000663


In [31]:
stop_point_pair_df = pd.concat(stop_point_pair_list, axis=0,ignore_index=True)

In [32]:
stop_point_pair_df.to_csv("stop_point_pair_vs_routeid.csv",index=False)

In [33]:
stop_point_pair_df.shape

(483218, 4)